In [1]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
import lightgbm as lgb
from workflow import main_indicators,Larry_simples_inverso
from sklearn.model_selection import train_test_split
warnings.filterwarnings("ignore")

In [2]:
ativo = 'win_1_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)

#df = df.iloc[:1000]

In [13]:
df = main_indicators(df)

df_acao = Larry_simples_inverso(df, tempo_fechar=-5, excluir_seguidos = True, acao = 'call')

In [14]:
df_acao = df_acao[df_acao['tend'] == 1]

In [15]:
compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: nan% -- Compra: 0.4450943897899495%
Venda: 0.0 -- Compra: 13755.0


In [16]:
df_new = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista', 'stop_real', 'tend'])
df_new = df_new.set_index(['Data','resultado_valor'])

df_new_sell = df_new[df_new['acao'] == 'call']
df_new_sell = df_new_sell.drop(columns = ['acao'])

In [17]:
X = df_new_sell.drop(columns = 'resultado_binario')
y = df_new_sell['resultado_binario']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

import lightgbm as lgb
clf = lgb.LGBMClassifier()
clf.fit(X_train, y_train)

# predict the results
y_pred=clf.predict(X_test)

# view accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

print('Training set score: {:.4f}'.format(clf.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(clf.score(X_test, y_test)))

LightGBM Model accuracy score: 0.5306
Training set score: 0.9821
Test set score: 0.5306


In [18]:
X_test['new'] = y_pred
a = X_test.reset_index()

a[a['new'] == 1]['resultado_valor'].sum()

2215.0

In [19]:
X_test

,,Open,High,Low,Close,Vol,MA_5,MA_21,SAR,BB_up,BB_mid,...,fastk,fastd,RSI,MOM,TRIX,CCI,ADX,MA_9,MA_100,new
Data,resultado_valor,,,,,,,,,,,,,,,,,,,,,
2022-05-13 09:27:00,35.0,107280.0,107395.0,107260.0,107380.0,26682,107340.0,107363.809524,122300.0,107437.619594,107355.25,...,85.714286,33.706564,55.205009,-5.0,0.010538,-2.500000e+01,21.080137,107349.655621,106984.55,0
2022-05-04 16:27:00,-50.0,108690.0,108845.0,108685.0,108820.0,26490,108777.0,108678.571429,122300.0,108868.016319,108693.75,...,72.549020,66.637900,60.736036,50.0,0.028630,8.477646e+01,27.520193,108731.247675,107499.75,1
2022-05-16 11:04:00,-135.0,108230.0,108350.0,108215.0,108350.0,46425,108233.0,108398.809524,122300.0,108627.650815,108388.25,...,77.142857,45.428386,48.150378,-185.0,0.005849,-5.427284e+01,19.748773,108323.072207,108231.25,1
2022-12-06 17:39:00,-125.0,109790.0,109890.0,109775.0,109880.0,25676,109800.0,109801.190476,96155.0,109874.521754,109801.25,...,93.548387,55.973032,56.189240,85.0,0.000633,1.481585e+02,19.115521,109804.369335,109789.10,1
2022-07-04 15:07:00,-25.0,100365.0,100380.0,100355.0,100380.0,4087,100364.0,100395.238095,122300.0,100477.633152,100394.25,...,84.615385,71.062271,48.449410,-20.0,0.000575,6.268517e-11,9.135539,100369.737984,100376.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01 16:42:00,-70.0,111605.0,111665.0,111605.0,111655.0,18698,111660.0,111629.285714,122300.0,111726.280366,111628.50,...,42.307692,44.172494,52.691933,85.0,0.005282,3.165736e+01,11.932084,111644.177298,111421.60,0
2022-08-16 11:11:00,-155.0,113175.0,113325.0,113175.0,113320.0,39641,113197.0,113240.952381,122300.0,113395.825814,113245.75,...,97.777778,63.905724,57.248618,155.0,0.003134,4.854586e+01,9.601287,113215.220682,113103.80,0
2022-09-05 16:46:00,-10.0,113620.0,113645.0,113610.0,113635.0,9910,113619.0,113641.190476,122300.0,113724.778170,113646.00,...,76.923077,48.748916,54.619324,10.0,0.001898,-5.907173e+00,19.272091,113626.150875,113537.35,0
